In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-3.2/transformers/3b-instruct/1/model.safetensors.index.json
/kaggle/input/llama-3.2/transformers/3b-instruct/1/config.json
/kaggle/input/llama-3.2/transformers/3b-instruct/1/model-00001-of-00002.safetensors
/kaggle/input/llama-3.2/transformers/3b-instruct/1/model-00002-of-00002.safetensors
/kaggle/input/llama-3.2/transformers/3b-instruct/1/README.md
/kaggle/input/llama-3.2/transformers/3b-instruct/1/USE_POLICY.md
/kaggle/input/llama-3.2/transformers/3b-instruct/1/tokenizer.json
/kaggle/input/llama-3.2/transformers/3b-instruct/1/tokenizer_config.json
/kaggle/input/llama-3.2/transformers/3b-instruct/1/LICENSE.txt
/kaggle/input/llama-3.2/transformers/3b-instruct/1/special_tokens_map.json
/kaggle/input/llama-3.2/transformers/3b-instruct/1/.gitattributes
/kaggle/input/llama-3.2/transformers/3b-instruct/1/generation_config.json


In [2]:
%%capture
%pip install -U transformers accelerate

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, TextStreamer
import torch


base_model = "/kaggle/input/llama-3.2/transformers/3b-instruct/1"

tokenizer = AutoTokenizer.from_pretrained(base_model)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    return_dict=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
if model.config.pad_token_id is None:
    model.config.pad_token_id = model.config.eos_token_id

In [5]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

Device set to use cuda:0


In [6]:
messages = [{"role": "user", "content": "Who is Neil Armstrong?"}]

prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

outputs = pipe(prompt, max_new_tokens=120, do_sample=True)

print(outputs[0]["generated_text"])

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Who is Neil Armstrong?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Neil Armstrong (1930-2012) was an American astronaut, engineer, and naval aviator who became the first person to walk on the Moon on July 20, 1969, during the Apollo 11 mission. He is widely regarded as one of the most iconic figures in the history of space exploration.

Early Life and Education:

Armstrong was born on August 5, 1930, in Wapakoneta, Ohio, to Stephen and Viola Armstrong. He grew up in a family of modest means and developed an interest in flight at an early age. Armstrong earned


In [7]:
from IPython.display import Markdown, display

messages = [
    {
        "role": "system",
        "content": "You are a skilled Python and Generative AI developer specializing in making AI Agents and writing agent specific prompts.",
    },
    {
        "role": "user",
        "content": "I want to use CrewAI to make AI agents , my idea is that i want two agents one can is retreival agent which will take user prompt , understand it and then store in vector database then the model will generate the answer and give to other agent which is authentication agent which will se whether the answer is according to question asked if yes then send to user make a crewai code for this.",
    },
]

prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

outputs = pipe(prompt, max_new_tokens=512, do_sample=True)

display(
    Markdown(
            outputs[0]["generated_text"].split(
                "<|start_header_id|>assistant<|end_header_id|>"
            )[1]
        )
    )



Here is a high-level CrewAI code that implements the two agents you described: Retrieval Agent and Authentication Agent.

**Please note that this is a basic implementation and may require modifications to suit your specific requirements.**

**Agent 1: Retrieval Agent**
```python
# Import required libraries
import crewai
from crewai import models
from crewai import utils

# Define the Retrieval Agent class
class RetrievalAgent:
    def __init__(self, model_name="distilbert-base-uncased"):
        self.model = crewai.models.get_model(model_name)

    def process_prompt(self, prompt):
        # Preprocess the prompt
        input_ids = self.model.encode(prompt)

        # Retrieve the vector representation
        vector = self.model.get_vector(input_ids)

        # Store the vector in the vector database
        utils.store_vector(vector, prompt)

        return vector

# Create a Retrieval Agent instance
retrieval_agent = RetrievalAgent()

# Define the Retrieval Agent's `run` method
def run_retrieval_agent(prompt):
    vector = retrieval_agent.process_prompt(prompt)
    return vector
```
**Agent 2: Authentication Agent**
```python
# Import required libraries
import crewai
from crewai import models
from crewai import utils

# Define the Authentication Agent class
class AuthenticationAgent:
    def __init__(self, model_name="distilbert-base-uncased"):
        self.model = crewai.models.get_model(model_name)

    def process_answer(self, answer, prompt):
        # Preprocess the answer and prompt
        input_ids = self.model.encode(answer)
        prompt_ids = self.model.encode(prompt)

        # Retrieve the vector representation of the answer
        answer_vector = self.model.get_vector(input_ids)

        # Compare the answer vector with the vector representation of the prompt
        similarity = self.model.compare_vectors(answer_vector, prompt_ids)

        # Return a boolean indicating whether the answer is correct
        return similarity > 0.5

# Create an Authentication Agent instance
authentication_agent = AuthenticationAgent()

# Define the Authentication Agent's `run` method
def run_authentication_agent(answer, prompt):
    return authentication_agent.process_answer(answer, prompt)
```
**Main Code**
```python
# Import required libraries
import crewai
from crewai import models
from crewai import utils

# Define the main function
def main():
    # Create a Retrieval Agent instance
    retrieval_agent = RetrievalAgent()

   